In [1]:
!pip install numpy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np

In [3]:
def sigmoid(z):
  return 1.0 / (1 + np.exp(-z))

In [4]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [5]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [6]:
def initialized_network(input_size: int, hidden_layers: list, output_size: int):
    layers = [input_size] + hidden_layers + [output_size]
    weights = []
    biases = []
    print(f' initialize_network {layers=}')
    
    l = len(layers) - 1

    for i in range(l):
        
        weight = np.random.uniform(-1, 1, (layers[i], layers[i+1]))
        bias = np.random.uniform(-1, 1, (layers[i+1],))
        print(f' #{i+1}/{l} {weight=}')
        print(f'   {bias=}\n')
        weights.append(weight)
        biases.append(bias)

    return weights, biases
        

In [7]:
def train_network(td, w, b, lr=0.1, e=1000):
    for epoch in range(e):
        total_error = 0
        for input_val, expected_output in td:
            total_error += train_once(input_val, expected_output, lr, w, b)
        if epoch % 1000 ==0:
            print(f"Epoch {epoch}, Total Error: {total_error}")

    return w, b
            

In [8]:
def forward_pass(input_data, weights, biases):
    activations = [input_data]
    for i in range(len(weights)):
        # on error: likely due to the input shape not matching the input nodes.
        input_data = np.dot(input_data, weights[i]) + biases[i]
        input_data = sigmoid(input_data)
        activations.append(input_data)
    return activations

In [9]:

def forward_pass_one(input_data, weights, biases):
    return forward_pass(input_data, weights, biases)[-1]


In [10]:
def train_once(input_val, expected_output, learning_rate, weights, biases):
    """
    # Example of adjusting training data to include negative example flag
    training_data = [
        # Positive examples: ((input_features), (target_output, False))
        ((input_features_for_example_1), (target_output_for_example_1, False)),
        # Negative examples: ((input_features), (target_output, True))
        ((input_features_for_negative_example_1), (target_output_for_negative_example_1, True)),
    ]

    """
    # Forward pass: Store activations for each layer
    activations = forward_pass(input_val, weights, biases)

    # Backward pass
    # Calculate output error
    output_error = expected_output - activations[-1]
    total_error = np.sum(output_error ** 2)

    # Calculate gradient for output layer
    d_error = output_error * sigmoid_derivative(activations[-1])

    for i in reversed(range(len(weights))):
        # Calculate error for the current layer
        d_weights = np.outer(activations[i], d_error)
        d_biases = d_error

        # Update weights and biases
        weights[i] += learning_rate * d_weights
        # cannot be: `biases[i] += learning_rate * d_biases` due to numpy
        biases[i] = biases[i] + (learning_rate * d_biases)

        # Propagate the error backward
        if i > 0:
            d_error = np.dot(d_error, weights[i].T) * sigmoid_derivative(activations[i])
    return total_error

In [11]:
def dictify(word):
    thedict = {}
    for i, letter in enumerate(word):
        thedict[letter] = (i+1)/len(word)
    return thedict

In [12]:
bits = dictify("catdog")
bits

{'c': 0.16666666666666666,
 'a': 0.3333333333333333,
 't': 0.5,
 'd': 0.6666666666666666,
 'o': 0.8333333333333334,
 'g': 1.0}

In [13]:
tds = [
    (
        (0.16666666666666666,0.3333333333333333, 0.5),
        (0.6666666666666666,0.8333333333333334, 1.0),
    ),
    (
        (0.6666666666666666,0.8333333333333334, 1.0),
        (0.16666666666666666,0.3333333333333333, 0.5),
    )
]

In [14]:
ws, bs = initialized_network(3,[3,5,3],3)
# train_network(tds, ws, bs)

 initialize_network layers=[3, 3, 5, 3, 3]
 #1/4 weight=array([[-0.64616989, -0.30221588,  0.67823204],
       [ 0.01190725, -0.57829623,  0.84202372],
       [-0.53739932,  0.54778629, -0.29282927]])
   bias=array([ 0.23474074,  0.46722634, -0.31571742])

 #2/4 weight=array([[ 0.61213502, -0.02923297, -0.99383848, -0.0167948 , -0.41833495],
       [-0.15798338,  0.88634282,  0.08192855,  0.52060837, -0.59161119],
       [ 0.39082894, -0.19624096,  0.12446626,  0.97816371, -0.22007961]])
   bias=array([ 0.57546402,  0.63472179, -0.54250525,  0.09514672,  0.93702812])

 #3/4 weight=array([[-0.21583292,  0.50918859, -0.20948805],
       [-0.47945152,  0.67891294,  0.33522201],
       [ 0.16967678, -0.73450437,  0.84246403],
       [ 0.18622179, -0.98146926, -0.81651222],
       [ 0.05899523,  0.35530216, -0.22825071]])
   bias=array([-0.79782263, -0.23662404,  0.98159396])

 #4/4 weight=array([[-0.9542908 ,  0.01028623,  0.2760435 ],
       [ 0.20863236, -0.59988059,  0.25852402],
      

In [15]:
cat_numbers = (bits["c"], bits["a"], bits["t"])
fp = forward_pass(cat_numbers, ws, bs)
stib = {v:k for k,v in bits.items()}
print(f"stib: {stib}")
print(f"fp: {fp}")

stib: {0.16666666666666666: 'c', 0.3333333333333333: 'a', 0.5: 't', 0.6666666666666666: 'd', 0.8333333333333334: 'o', 1.0: 'g'}
fp: [(0.16666666666666666, 0.3333333333333333, 0.5), array([0.46563307, 0.62198563, 0.48290197]), array([0.7213088 , 0.74604661, 0.29024263, 0.70754587, 0.56659865]), array([0.25034681, 0.48272028, 0.64977543]), array([0.34679745, 0.64112466, 0.71874695])]


In [16]:
def closest_match(value, values_in, values_out):
    fp = forward_pass(value, ws, bs)
    for i in range(len(ws)):
        value = np.dot(value, ws[i]) + bs[i]
        value = sigmoid(value)
    final_output = fp[0]
    closest_numb = min(values_in.values(), key=lambda x: abs(x - final_output))
    predicted_letter = values_out[closest_numb]
    return predicted_letter

In [17]:

def predict(input_val, weights, biases, letter_to_num, num_to_letter):
    # Ensure input is in the correct format (numpy array)
    input_val = np.array(input_val)

    v = forward_pass_one(input_val, weights, biases)
    # Generalized forward pass through all layers
    for i in range(len(weights)):
        input_val = np.dot(input_val, weights[i]) + biases[i]
        input_val = sigmoid(input_val)
    # assert v == input_val
    # Convert the final output to the closest letter
    # Here, we're assuming the output is a single number. Adjust as needed for multiple outputs.
    # final_output = v[0]  # Assuming single output for simplicity
    res = ''
    for final_output in v:
        closest_num = min(letter_to_num.values(), key=lambda x: abs(x - final_output))
        predicted_letter = num_to_letter[closest_num]
        # print(predicted_letter)
        res += predicted_letter
    return res


In [18]:
ws, bs

([array([[-0.64616989, -0.30221588,  0.67823204],
         [ 0.01190725, -0.57829623,  0.84202372],
         [-0.53739932,  0.54778629, -0.29282927]]),
  array([[ 0.61213502, -0.02923297, -0.99383848, -0.0167948 , -0.41833495],
         [-0.15798338,  0.88634282,  0.08192855,  0.52060837, -0.59161119],
         [ 0.39082894, -0.19624096,  0.12446626,  0.97816371, -0.22007961]]),
  array([[-0.21583292,  0.50918859, -0.20948805],
         [-0.47945152,  0.67891294,  0.33522201],
         [ 0.16967678, -0.73450437,  0.84246403],
         [ 0.18622179, -0.98146926, -0.81651222],
         [ 0.05899523,  0.35530216, -0.22825071]]),
  array([[-0.9542908 ,  0.01028623,  0.2760435 ],
         [ 0.20863236, -0.59988059,  0.25852402],
         [ 0.73787809,  0.55133482,  0.06066102]])],
 [array([ 0.23474074,  0.46722634, -0.31571742]),
  array([ 0.57546402,  0.63472179, -0.54250525,  0.09514672,  0.93702812]),
  array([-0.79782263, -0.23662404,  0.98159396]),
  array([-0.97440882,  0.5090044 ,  0

In [19]:
closest_match(.1, bits, stib)

'c'

In [20]:
fp

[(0.16666666666666666, 0.3333333333333333, 0.5),
 array([0.46563307, 0.62198563, 0.48290197]),
 array([0.7213088 , 0.74604661, 0.29024263, 0.70754587, 0.56659865]),
 array([0.25034681, 0.48272028, 0.64977543]),
 array([0.34679745, 0.64112466, 0.71874695])]

In [21]:
for letter in fp[0]:
    print(closest_match(letter, bits, stib))

c
a
t


In [22]:
for letter in fp[2]:
    print(closest_match(letter, bits, stib))

d
d
a
d
t


In [23]:
train_network(tds, ws, bs)

Epoch 0, Total Error: 0.39704180878778755


([array([[-0.61608676, -0.31818543,  0.71262477],
         [ 0.04179725, -0.59514755,  0.8740042 ],
         [-0.50770245,  0.53005321, -0.26326103]]),
  array([[ 0.60845023, -0.00158743, -0.96120404, -0.07475767, -0.4338791 ],
         [-0.15647963,  0.89669424,  0.08244999,  0.48653833, -0.59526832],
         [ 0.40072793, -0.2253561 ,  0.06535446,  1.0117458 , -0.19931017]]),
  array([[-0.22455019,  0.54441835, -0.18787186],
         [-0.4969951 ,  0.71445752,  0.37684339],
         [ 0.1914931 , -0.71693672,  0.79301611],
         [ 0.20293098, -0.94394543, -0.8532125 ],
         [ 0.06391814,  0.38453348, -0.23820579]]),
  array([[-0.93802254, -0.0450757 ,  0.27926059],
         [ 0.35906052, -0.59175389,  0.35684661],
         [ 0.82891844,  0.45159293,  0.10647435]])],
 [array([ 0.23358196,  0.46193575, -0.33019088]),
  array([ 0.58107114,  0.63768957, -0.56351716,  0.06236376,  0.93980462]),
  array([-0.80686018, -0.18734129,  1.00442149]),
  array([-0.83564076,  0.35642907,  0

In [24]:
fp

[(0.16666666666666666, 0.3333333333333333, 0.5),
 array([0.46563307, 0.62198563, 0.48290197]),
 array([0.7213088 , 0.74604661, 0.29024263, 0.70754587, 0.56659865]),
 array([0.25034681, 0.48272028, 0.64977543]),
 array([0.34679745, 0.64112466, 0.71874695])]

In [25]:
result = None
count = 0

while result != "dog" or count <= 10:
    count += 1
    train_network(tds, ws, bs)
    result = predict(cat_numbers, ws, bs, bits, stib)

Epoch 0, Total Error: 0.37774737872052205
Epoch 0, Total Error: 0.37662561185042076
Epoch 0, Total Error: 0.3747328504250761
Epoch 0, Total Error: 0.370391462445424
Epoch 0, Total Error: 0.35527712208193485
Epoch 0, Total Error: 0.2523060918949412
Epoch 0, Total Error: 0.02337137588066681
Epoch 0, Total Error: 0.005892195963310244
Epoch 0, Total Error: 0.0032144960288192238
Epoch 0, Total Error: 0.0021093668167509647
Epoch 0, Total Error: 0.001529925827544427


In [26]:
predict(cat_numbers, ws, bs, bits, stib)

'dog'

In [210]:
from importlib import reload
reload(training_data)

<module 'training_data' from '/Users/calebstripes/Code/AIML/aibrain/training_data.py'>

In [211]:
import training_data
training_data.setup()

{'a': 0.018867924528301886,
 'b': 0.03773584905660377,
 'c': 0.05660377358490566,
 'd': 0.07547169811320754,
 'e': 0.09433962264150944,
 'f': 0.11320754716981132,
 'g': 0.1320754716981132,
 'h': 0.1509433962264151,
 'i': 0.16981132075471697,
 'j': 0.18867924528301888,
 'k': 0.20754716981132076,
 'l': 0.22641509433962265,
 'm': 0.24528301886792453,
 'n': 0.2641509433962264,
 'o': 0.2830188679245283,
 'p': 0.3018867924528302,
 'q': 0.32075471698113206,
 'r': 0.33962264150943394,
 's': 0.3584905660377358,
 't': 0.37735849056603776,
 'u': 0.39622641509433965,
 'v': 0.41509433962264153,
 'w': 0.4339622641509434,
 'x': 0.4528301886792453,
 'y': 0.4716981132075472,
 'z': 0.49056603773584906,
 'A': 0.5094339622641509,
 'B': 0.5283018867924528,
 'C': 0.5471698113207547,
 'D': 0.5660377358490566,
 'E': 0.5849056603773585,
 'F': 0.6037735849056604,
 'G': 0.6226415094339622,
 'H': 0.6415094339622641,
 'I': 0.660377358490566,
 'J': 0.6792452830188679,
 'K': 0.6981132075471698,
 'L': 0.7169811320754

In [212]:
from pprint import pprint as pp
pp(training_data.DATA)

(('cookie', 'cream '),
 ('peanut', 'jelly '),
 ('salt  ', 'pepper'),
 ('mac   ', 'cheese'),
 ('bread ', 'butter'),
 ('fish  ', 'chips '),
 ('ham   ', 'eggs  '),
 ('bacon ', 'eggs  '),
 ('milk  ', 'cereal'))


In [213]:
_bits = training_data.bits
cookie = [_bits[x] for x in training_data.DATA[0][0]]
cream = [_bits[x] for x in training_data.DATA[0][1]]
print(cookie)
print(cream)
train_once(cookie, cream, 0.1, ws2, bs2)
# def train_once(input_val, expected_output, learning_rate, weights, biases):

[0.05660377358490566, 0.2830188679245283, 0.2830188679245283, 0.20754716981132076, 0.16981132075471697, 0.09433962264150944]
[0.05660377358490566, 0.33962264150943394, 0.09433962264150944, 0.018867924528301886, 0.24528301886792453, 1.0]


np.float64(4.2999927781022e-05)

In [267]:
ws2, bs2 = initialized_network(6,[7, 9, 7],6)

 initialize_network layers=[6, 7, 9, 7, 6]
 #1/4 weight=array([[ 0.91471865,  0.44200208,  0.82441172,  0.82971916, -0.19088331,
         0.31346112, -0.30526574],
       [ 0.82612182,  0.70708893, -0.6864128 , -0.60650575, -0.17663885,
        -0.04491059, -0.90719727],
       [-0.37153297,  0.16553862,  0.18097318,  0.78694747, -0.64339694,
        -0.12473057, -0.69311072],
       [ 0.20230783, -0.81658025, -0.90992879, -0.28191206, -0.71159261,
        -0.05836455, -0.53694171],
       [ 0.61887206,  0.98099525, -0.16547586, -0.36691473,  0.55713656,
         0.68337213,  0.03705954],
       [ 0.4276622 , -0.4773619 ,  0.35273551, -0.29439251,  0.4491304 ,
         0.0502293 ,  0.12923663]])
   bias=array([ 0.9454774 , -0.76256776,  0.07646674,  0.99623775,  0.59634668,
        0.39034356, -0.22900036])

 #2/4 weight=array([[ 0.1037078 ,  0.84915698,  0.2270249 , -0.68764245, -0.54038224,
        -0.633058  ,  0.51225188,  0.09379401,  0.57438782],
       [-0.59531049, -0.39122926,

In [268]:
num_data = training_data.convert_data_to_floats(training_data.DATA)

In [311]:
result = None
count = 0
success = False

while success is False or count <= 10:
    count += 1
    train_network(num_data, ws2, bs2)
    result = predict(training_data.get_word("milk  "), ws2, bs2, training_data.bits, training_data.stib)
    success = result == "cereal"
    if success is True: break
    print(f"'{result}'")

Epoch 0, Total Error: 1.0825432815648142
'ceqfal'
Epoch 0, Total Error: 0.0064980965674865596
'cerfal'
Epoch 0, Total Error: 0.0036616063752657707


In [322]:
training_data.get_word("cookie")

(0.05660377358490566,
 0.2830188679245283,
 0.2830188679245283,
 0.20754716981132076,
 0.16981132075471697,
 0.09433962264150944)

In [270]:
print(training_data.bits)

{'a': 0.018867924528301886, 'b': 0.03773584905660377, 'c': 0.05660377358490566, 'd': 0.07547169811320754, 'e': 0.09433962264150944, 'f': 0.11320754716981132, 'g': 0.1320754716981132, 'h': 0.1509433962264151, 'i': 0.16981132075471697, 'j': 0.18867924528301888, 'k': 0.20754716981132076, 'l': 0.22641509433962265, 'm': 0.24528301886792453, 'n': 0.2641509433962264, 'o': 0.2830188679245283, 'p': 0.3018867924528302, 'q': 0.32075471698113206, 'r': 0.33962264150943394, 's': 0.3584905660377358, 't': 0.37735849056603776, 'u': 0.39622641509433965, 'v': 0.41509433962264153, 'w': 0.4339622641509434, 'x': 0.4528301886792453, 'y': 0.4716981132075472, 'z': 0.49056603773584906, 'A': 0.5094339622641509, 'B': 0.5283018867924528, 'C': 0.5471698113207547, 'D': 0.5660377358490566, 'E': 0.5849056603773585, 'F': 0.6037735849056604, 'G': 0.6226415094339622, 'H': 0.6415094339622641, 'I': 0.660377358490566, 'J': 0.6792452830188679, 'K': 0.6981132075471698, 'L': 0.7169811320754716, 'M': 0.7358490566037735, 'N': 0.

In [271]:
def query(value: str):
    value = value.ljust(6)
    result = predict(training_data.get_word(value), ws2, bs2, training_data.bits, training_data.stib)
    return result

In [312]:
print(query("bacon"))
print(query("milk"))

eggsZ 
cereal


In [321]:
# ws2 is weights, bs2 is biases. We are going to manually edit these below
#   0.73409465
# ws2[1][0][1] = 0.74
# ws2[2][5][2] = 2.9
# ws2[3][3][3] = 0.59
# ws2[1][5][2] = 1.0
np.delete(ws2, (0))
# print(bs2)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.